In [1]:
#imports

In [2]:
# 1st variable - Start cycle
def start_cycle_random(dataset):
    raise NotImplementedError

In [3]:
# 1st variable - Start cycle
def starty_cycle_heuristic(dataset):
    raise NotImplementedError

In [5]:
# 2nd variable - Find more optimal solution
# switch parameter defines how u find neighbour solutions (is it change_vertices or change_edges func)
def greedy_search(cycleA, cycleB, switch):
    raise NotImplementedError

In [6]:
# 2nd variable - Find more optimal solution
def steep_search(cycleA, cycleB):
    raise NotImplementedError

In [7]:
# 3rd variable - Generate neighbour solution
#changes vertex internally or externaly between cycles
def change_vertices():
    raise NotImplementedError

In [8]:
# 3rd variable - Generate neighbour solution
#changes vertexes externally between cycles or edges internally
def change_edges():
    raise NotImplementedError

In [9]:
# algorytm losowego bładzenia który robi losowo wybrany ruch jako punkt odniesienia wzgledem rozwiazan steep search i greedy search
def random_wandering():
    raise NotImplementedError

SyntaxError: unexpected EOF while parsing (<ipython-input-9-60495213414f>, line 2)

In [10]:
#MAIN - tu poskladac wyzej zaimplementowane funkcje w 8 roznych kombinacji algorytmow i sprawdzic ich czasy i wyplotowac skutecznosci